In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from collections import OrderedDict
from itertools import chain

In [24]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [25]:
batch1 = 'cifar-10-batches-py/data_batch_1'
batch2 = 'cifar-10-batches-py/data_batch_2'
batch3 = 'cifar-10-batches-py/data_batch_3'
batch4 = 'cifar-10-batches-py/data_batch_4'
batch5 = 'cifar-10-batches-py/data_batch_5'


In [26]:
dict1 = unpickle(batch1)
dict2 = unpickle(batch2)
dict3 = unpickle(batch3)
dict4 = unpickle(batch4)
dict5 = unpickle(batch5)
train = defaultdict(list)

